In [ ]:
import pyarrow.parquet as pq
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from upsetplot import plot
from setup_plot import setup_local, get_colors, get_markers
import matplotlib.patches as mpatches
from matplotlib.patches import Patch

import os
import duckdb

SET1 = "ldms_set1.parquet"
SET2 = "ldms_set2.parquet"

THRESH  = 0.005
GPU_UTIL_COL = "nersc_ldms_dcgm_gpu_utilization"

needed = [
    "JobID",
    "nersc_ldms_dcgm_tensor_active",
    "nersc_ldms_dcgm_fp64_active",
    "nersc_ldms_dcgm_fp32_active",
    "nersc_ldms_dcgm_fp16_active",
    GPU_UTIL_COL,
]

def list_counters(fname: str) -> list[str]:
    con = duckdb.connect()
    try:
        cols = con.execute(f"DESCRIBE SELECT * FROM parquet_scan('{fname}')").fetchall()
    finally:
        con.close()
    return [c[0] for c in cols if c[0].startswith("nersc_ldms_dcgm_")]

def avg_expr(cols: list[str], overlap: set[str], suffix_overlaps: bool = False) -> str:
    parts = []
    for c in cols:
        alias = f"{c}_s2" if (suffix_overlaps and c in overlap) else c
        parts.append(f"avg({c}) AS {alias}")
    return ",\n       ".join(parts)

def build_job_means_all():
    counters1 = list_counters(SET1)
    counters2 = list_counters(SET2)
    overlap = set(counters1).intersection(counters2)

    con = duckdb.connect()
    con.execute("PRAGMA memory_limit='15GB';")
    con.execute(f"PRAGMA threads={os.cpu_count() or 1};")

    df = con.execute(f"""
      WITH
      s1_gpu AS (
        SELECT
          JobID::VARCHAR AS JobID,
          hostname,
          gpu_id,
          {avg_expr(counters1, overlap, suffix_overlaps=False)}
        FROM parquet_scan('{SET1}')
        GROUP BY JobID, hostname, gpu_id
      ),
      s1_job AS (
        SELECT
          JobID,
          {avg_expr(counters1, overlap, suffix_overlaps=False)}
        FROM s1_gpu
        GROUP BY JobID
      ),
      s2_gpu AS (
        SELECT
          JobID::VARCHAR AS JobID,
          hostname,
          gpu_id,
          {avg_expr(counters2, overlap, suffix_overlaps=False)}
        FROM parquet_scan('{SET2}')
        GROUP BY JobID, hostname, gpu_id
      ),
      s2_job AS (
        SELECT
          JobID,
          {avg_expr(counters2, overlap, suffix_overlaps=True)}
        FROM s2_gpu
        GROUP BY JobID
      )
      SELECT *
      FROM s1_job AS s1
      LEFT JOIN s2_job AS s2 USING (JobID)
    """).df()

    con.close()
    return df

setup_local()
colors  = get_colors()

df = build_job_means_all()[needed]
df["JobID"] = df["JobID"].astype(str)

df["TNSR_ACTV"] = df["nersc_ldms_dcgm_tensor_active"] > THRESH
df["FP64_ACTV"] = df["nersc_ldms_dcgm_fp64_active"]   > THRESH
df["FP32_ACTV"] = df["nersc_ldms_dcgm_fp32_active"]   > THRESH
df["FP16_ACTV"] = df["nersc_ldms_dcgm_fp16_active"]   > THRESH

pattern_cols = ["TNSR_ACTV", "FP64_ACTV", "FP32_ACTV", "FP16_ACTV"]

df = df[df[pattern_cols].any(axis=1)].copy()

DESIRED = [
    (False, False, True,  False),
    (True,  False, True,  False),
    (False, True,  False, False),
    (True,  True,  False, False),
    (True,  False, False, False),
]

counts_series = df.groupby(pattern_cols, observed=True).size()
mean_series = df.groupby(pattern_cols, observed=True)[GPU_UTIL_COL].mean()
mean_util_pct = mean_series.round(2).to_dict()

desired_present = [p for p in DESIRED if p in counts_series.index]
rest_sorted    = sorted([p for p in counts_series.index if p not in desired_present],
                        key=lambda p: counts_series[p], reverse=True)
new_index = desired_present + rest_sorted

counts_ordered = (
    counts_series
    .reindex(new_index)
)

fig = plt.figure(figsize=(10, 6))
plot_result = plot(
    counts_ordered,
    sort_by=None,
    sort_categories_by=None,
    fig=fig,
    element_size=None,
    show_percentages=False,
    facecolor=colors[2],
    min_subset_size=1000
)

ax1 = plot_result["intersections"]
ax1.set_yticks([0, 8000, 16000, 24000, 32000, 40000])
ax1.set_ylabel("Number of jobs", fontsize=25)
for txt in list(ax1.texts):
    txt.set_visible(False)

ax2 = ax1.twinx()
ax2.set_ylim(0, 100)
ax2.set_yticks([0, 20, 40, 60, 80, 100])
ax2.set_ylabel("Mean of GPU_UTIL (%)", fontsize=25)

patterns_displayed = [idx for idx in counts_ordered.index if counts_ordered.loc[idx] >= 1000]

left_bars = [p for p in ax1.patches if isinstance(p, mpatches.Rectangle) and p.get_height() > 0]
left_bars = sorted(left_bars, key=lambda r: r.get_x())

for i, rect in enumerate(left_bars[:len(patterns_displayed)]):
    pattern = patterns_displayed[i]

    mu_pct = mean_util_pct.get(pattern)
    if mu_pct is None:
        continue

    x = rect.get_x()
    w = rect.get_width()
    rect.set_width(w * 0.5)
    ax2.bar(x + 0.75*w, mu_pct, width=w*0.5,
            color=colors[0], edgecolor="black",
            label="Mean of GPU_UTIL (%)" if i == 0 else None)

handles = [
    Patch(facecolor=colors[2], edgecolor="black", label="Number of jobs"),
    Patch(facecolor=colors[0], edgecolor="black", label="Mean (of GPU_UTIL) (%)")
]
ax1.legend(
    handles=handles,
    loc="upper right",
    bbox_to_anchor=(1.0, 1.13),
    fontsize=18,
    framealpha=0.7,
    borderpad=0.1,
    columnspacing=0.2
)

plot_result["totals"].set_xlabel("Number of jobs", fontsize=23)

ax_intersections = plot_result["intersections"]
ax_totals        = plot_result["totals"]
ax_matrix        = plot_result["matrix"]
ax_intersections.tick_params(axis='both', labelsize=22)
ax_totals.tick_params(axis='both', labelsize=18)
ax_matrix.tick_params(axis='y', labelsize=20)
ax2.tick_params(labelsize=22)

plt.suptitle(
    "Number and mean of GPU_UTIL of jobs\nthat use different GPU FP pipelines",
    fontsize=24, y=1.07, x=0.6
)
plt.show()
